In [1]:
from fastembed import TextEmbedding

In [2]:
import numpy as np
import pandas as pd

### Q1. Embedding the Query

In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [4]:
query = 'I just discovered the course. Can I join now?'
sz = 512

In [5]:
model = TextEmbedding(model_name=model_handle)

In [6]:
model.embed(documents=query, batch_size=sz)

<generator object TextEmbedding.embed at 0x75133ba066b0>

In [7]:
embedding_generator = model.embed(documents=query, batch_size=sz)

In [8]:
q = list(embedding_generator)[0]

In [9]:
min(q)

np.float64(-0.11726373885183883)

In [10]:
min(q).round(3)

np.float64(-0.117)

In [11]:
max(q).round(3)

np.float64(0.133)

#### Cosine Similarity

In [12]:
np.linalg.norm(q)

np.float64(1.0)

In [13]:
q.dot(q)

np.float64(1.0000000000000002)

### Q2. Cosine similarity with another vector

In [14]:
doc = 'Can I still join the course after the start date?'

In [15]:
embedding_generator = model.embed(documents=doc, batch_size=512)
x = list(embedding_generator)[0]

In [16]:
q.dot(x)

np.float64(0.9008528895674548)

### Q3. Ranking by cosine

In [17]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [18]:
for i, doc in enumerate(documents, 0):
    embedding_generator = model.embed(doc['text'])
    embedding_list = list(embedding_generator)[0]

    cosine = q.dot(embedding_list)

    print(f'{i}: {cosine}')

0: 0.762968451872193
1: 0.8182378156620136
2: 0.8085397445747489
3: 0.7133078832064158
4: 0.7304499196411823


In [19]:
embeds = []

for i, doc in enumerate(documents, 0):
    embedding_generator = model.embed(doc['text'])
    embedding_list = list(embedding_generator)[0]

    embeds.append(embedding_list)

embeds = np.array(embeds)
print(embeds.dot(q))

[0.76296845 0.81823782 0.80853974 0.71330788 0.73044992]


### Q4. Ranking by cosine, version two

In [20]:
for i, doc in enumerate(documents, 0):
    full_text = doc['question'] + ' ' + doc['text']
    
    embedding_generator = model.embed(full_text)
    embedding_list = list(embedding_generator)[0]

    cosine = q.dot(embedding_list)

    print(f'{i}: {cosine}')

0: 0.851454319443226
1: 0.8436594005975433
2: 0.8408287224005012
3: 0.7755157657626952
4: 0.8086007917931166


### Q5. Selecting the embedding model

In [21]:
sizes = set()

for model in TextEmbedding.list_supported_models():
    sizes.add(model['dim'])

print(min(sizes))

384


### Q6. Indexing with qdrant

In [22]:
import requests

In [23]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

In [24]:
documents = []

for course in docs_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
from qdrant_client import QdrantClient, models

In [26]:
qd_client = QdrantClient('http://localhost:7333/')

In [27]:
qd_client.create_collection(
    collection_name="assignment-2",
    vectors_config=models.VectorParams(
        size=512,
        distance=models.Distance.COSINE
    ),
)

True

In [28]:
import uuid

qd_client.upsert(
    collection_name="assignment-2",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector=models.Document(
                text=doc['question'] + ' ' + doc['text'],
                model=model_handle
            ),
            payload=doc
        )

        for doc in documents
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
search_results = qd_client.query_points(
    collection_name="assignment-2",
    query=models.Document(
        text = query,
        model = model_handle
    ),
    with_payload=True
)

In [30]:
for result in search_results.points:
    print(f'Score: {result.score}')
    print(f'Payload:\n{result.payload}')
    print("---------")

Score: 0.8620738
Payload:
{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}
---------
Score: 0.83894074
Payload:
{'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was